In [222]:
import pandas as pd
import csv
import json

In [223]:
height = 730
width = 600

fields = []

In [224]:
def generateDf(filename):
    jsonObj = json.load(open(filename, 'r'))
    newJson = jsonObj[2]['Label']['objects']
    df = pd.DataFrame(columns=['top','left','height','width','type'])
    for data in newJson:
        tmp = data['bbox']
        tmp['type']=data['title']
        tmp['value']=data['value']
        df = df.append(tmp,ignore_index=True)
    df['group']='NaN'
    return df

df = generateDf('exported_json.json')
df

,top,left,height,width,type,value,group
0,362,1451,65,131,label,date,NaN
1,187,1068,83,401,label,title,NaN
2,580,292,56,148,label,name,NaN
3,741,292,69,161,label,phone,NaN
4,749,1151,61,139,label,email,NaN
5,950,301,57,1041,label,checkbox title,NaN
6,1072,445,70,104,label,yes,NaN
7,1081,724,56,82,label,no,NaN
8,1089,1002,61,258,label,somewhat,NaN
9,1220,288,70,257,label,comments,NaN


In [225]:
# structure = -1   # 0: horizontal (Fields are beside labels)  1: vertical  -1: not set
# i = 0; j=0


# def isFieldInRight(i,j):
#     if(labelsdf['left'][i]+labelsdf['width'][i] <= fieldsdf['left'][j]):
#         if(fieldsdf['top'][j]+fieldsdf['height'][j] >= labelsdf['top'][i]+labelsdf['height'][i] and fieldsdf['top'][j] <= labelsdf['top'][i]+labelsdf['height'][i]):
#             # field is found at the right side with proper place
#             return True
#     return False

# def isFieldBelow(i,j):
#     if(labelsdf['top'][i]+labelsdf['height'][i] <= fieldsdf['top'][j]):
#         if(fieldsdf['left'][j] <= labelsdf['left'][i] and fieldsdf['left'][j]+fieldsdf['width'][j] >= labelsdf['left'][i]+labelsdf['width'][i]):
#             # field is found below 
#             return True
#     return False

# def isLabelInRight(i):
#     if(i<labelsdf.count-1):
#         if(labelsdf['left'][i]+labelsdf['width'][i] < labelsdf['left'][i+1]):
#             if(labelsdf['top'][i]+labelsdf['height'][i] >= labelsdf['top'][i+1] and labelsdf['top'][i+1]+labelsdf['height'][i+1] > labelsdf['top'][i]):
#                 # label is found at the right side with proper place
#                 return True
#     return False
    
# def isLabelBelow(i):
#     if(i<labelsdf.count-1):
#         if(labelsdf['top'][i]+labelsdf['height'][i] < labelsdf['top'][i+1]):
#             if(labelsdf['left'][i] <= labelsdf['left'][i+1]+labelsdf['width'][i+1] and labelsdf['left'][i]+labelsdf['width'][i] > labelsdf['left'][i+1]):
#                 # field is found below 
#                 return True
#     return False
    

# def decideStructure(i,j):
#     if(isFieldInRight(i,j) and isLabelBelow(i,j)):
#         structure=0
#     if(isLabelBelow(i,j) and isFieldInRight(i,j)):
#         structure=1

# def elementsInRows(i,j):
#     labels=0;fields=0
#     while(labelsdf['left'][i]+labelsdf['width'][i]<600):
        

In [226]:
#ERROR   ( max_field_height - current_height )/2  strip_top = current_top - ERROR   strip_bottm = current_bottom + ERRROR

In [227]:
df = df.sort_values(by=['top']) #.reset_index(drop=True)
df

,top,left,height,width,type,value,group
1,187,1068,83,401,label,title,NaN
10,344,1665,96,614,field,date,NaN
0,362,1451,65,131,label,date,NaN
11,536,501,122,1351,field,name,NaN
2,580,292,56,148,label,name,NaN
13,715,1316,104,854,field,email,NaN
12,719,497,100,606,field,phone,NaN
3,741,292,69,161,label,phone,NaN
4,749,1151,61,139,label,email,NaN
5,950,301,57,1041,label,checkbox title,NaN


In [228]:
min_field_height = df['height'].min()
print(min_field_height)
max_field_height = df['height'].max()
print(max_field_height)

def assign_from_last(curr_df):
    in_strip_elements=curr_df.shape[0]
    
    if(in_strip_elements%2==1):
        # set group of all elements after first,index(startIndex) of first
        startIndex = curr_df.index[0]
        parent_group = startIndex
        
        if curr_df.iloc[0].type=='label':  
            if curr_df.iloc[1].type=='field': 
                prevIndex=None
                for index,row in curr_df.iloc[1:].iterrows():
                    if row.type=='label':
                        df['group'][prevIndex]=[start,index]
                    if row.type=='field':
                        prevIndex=index
                        
            if curr_df.iloc[1].type=='label': 
                prevIndex=curr_df.index[1]
                for index,row in curr_df.iloc[1:].iterrows():
                    if row.type=='label':
                        prevIndex=index
                    if row.type=='field':
                        df['group'][index]=[start,prevIndex]

        else:
            pass
            # this should not be the case 
                
            
    else:
            if curr_df.iloc[0].type=='field': 
                prevIndex=curr_df.index[0]
                print()
                for index,row in curr_df.iloc[1:].iterrows():
                    if row.type=='label':
                        df['group'][prevIndex]=index
                    if row.type=='field':
                        prevIndex=index

            if curr_df.iloc[0].type=='label': 
                prevIndexValue=curr_df.index[1]
                for index,row in curr_df.iloc[1:].iterrows():
                    print("index:",index,'row:\n',row.to_frame().T)
                    if row.type=='label':
                        prevIndexValue=index
                    if row.type=='field':
                        df['group'][index]=prevIndexValue

element=-1
parent_group=None
# print(df.shape[0])
while(element < df.shape[0]-1):
    element+=1;labels=0;fields=0
    in_strip_elements=0
    topy=0;bottomy=0
    ERROR = (max_field_height - df['height'][element] )/2
    topy=df['top'][element] - ERROR
    bottomy=df['height'][element]+df['top'][element] + ERROR
#     print('top:',topy,'bottom:',bottomy)
    curr_df = df[(df.top>=topy) & (df.top+df.height<=bottomy)]
    curr_df=curr_df.sort_values(by='left') # .reset_index(drop=True)
#     print(curr_df)
    for i in range(curr_df.shape[0]):
        element+=1
        in_strip_elements+=1
        if curr_df.iloc[i].type=='label':
            labels+=1
        if curr_df.iloc[i].type=='field':
            fields+=1            
    element-=1
    
#     if element==1:
#         if df.type=='label':
#             parent_group=element-1
#             # set parent_group to all elements in below strip 
#         if df.type=='field':
#             pass
#             # inherit parent_group from above strip
        
    if (labels>0 and fields>0):
        assign_from_last(curr_df)
        
print('Original DF:\n',df)


56
213
index: 10 row:
     top  left height width   type value group
10  344  1665     96   614  field  date   NaN
index: 11 row:
     top left height width   type value group
11  536  501    122  1351  field  name   NaN
index: 12 row:
     top left height width   type  value group
12  719  497    100   606  field  phone   NaN
index: 4 row:
    top  left height width   type  value group
4  749  1151     61   139  label  email   NaN
index: 13 row:
     top  left height width   type  value group
13  715  1316    104   854  field  email   NaN
Original DF:
      top  left height width      type           value group
1    187  1068     83   401     label           title   NaN
10   344  1665     96   614     field            date    10
0    362  1451     65   131     label            date   NaN
11   536   501    122  1351     field            name    11
2    580   292     56   148     label            name   NaN
13   715  1316    104   854     field           email     4
12   719   497    10